# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f93d009feb0>
├── 'a' --> tensor([[-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865]])
└── 'x' --> <FastTreeValue 0x7f93d009fe80>
    └── 'c' --> tensor([[ 1.8700,  1.2475, -2.5124,  0.7783],
                        [ 0.7391,  0.2616,  0.2534, -0.1498],
                        [ 0.4851,  1.7314, -0.3190,  1.2785]])

In [4]:
t.a

tensor([[-0.1372,  0.0257, -0.1627],
        [ 0.2138, -0.8140,  0.1865]])

In [5]:
%timeit t.a

63.8 ns ± 0.839 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f93d009feb0>
├── 'a' --> tensor([[-0.9107,  1.6172,  0.5920],
│                   [-0.8755, -1.0268, -1.0591]])
└── 'x' --> <FastTreeValue 0x7f93d009fe80>
    └── 'c' --> tensor([[ 1.8700,  1.2475, -2.5124,  0.7783],
                        [ 0.7391,  0.2616,  0.2534, -0.1498],
                        [ 0.4851,  1.7314, -0.3190,  1.2785]])

In [7]:
%timeit t.a = new_value

64 ns ± 1.76 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865]]),
    x: Batch(
           c: tensor([[ 1.8700,  1.2475, -2.5124,  0.7783],
                      [ 0.7391,  0.2616,  0.2534, -0.1498],
                      [ 0.4851,  1.7314, -0.3190,  1.2785]]),
       ),
)

In [10]:
b.a

tensor([[-0.1372,  0.0257, -0.1627],
        [ 0.2138, -0.8140,  0.1865]])

In [11]:
%timeit b.a

60.9 ns ± 2.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9891,  0.3202, -0.5176],
               [-0.2296, -0.3677,  0.1601]]),
    x: Batch(
           c: tensor([[ 1.8700,  1.2475, -2.5124,  0.7783],
                      [ 0.7391,  0.2616,  0.2534, -0.1498],
                      [ 0.4851,  1.7314, -0.3190,  1.2785]]),
       ),
)

In [13]:
%timeit b.a = new_value

548 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

909 ns ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.3 µs ± 270 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

244 µs ± 8.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

244 µs ± 8.96 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f93d0092250>
├── 'a' --> tensor([[[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]],
│           
│                   [[-0.1372,  0.0257, -0.1627],
│                    [ 0.2138, -0.8140,  0.1865]]])
└── 'x' --> <FastTreeValue 0x7f93d0092190>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

40.7 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f93d0092220>
├── 'a' --> tensor([[-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865],
│                   [-0.1372,  0.0257, -0.1627],
│                   [ 0.2138, -0.8140,  0.1865]])
└── 'x' --> <FastTreeValue 0x7f93d00927f0>
    └── 'c' --> tensor([[ 1.8700,  1.2475, -2.5124,  0.7783],
                        [ 0.7391,  0.2616,  0.2534, -0.1498],
                 

In [23]:
%timeit t_cat(trees)

37.3 µs ± 936 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

71.3 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]],
       
               [[-0.1372,  0.0257, -0.1627],
                [ 0.2138, -0.8140,  0.1865]]]),
    x: Batch(
           c: tensor([[[ 1.8700,  1.2475, -2.5124,  0.7783],
                       [ 0.7391,  0.2616,  0.2534, -0.1498],
                       [ 0.4851,  1.7314, -0.3190,  1.2785]],
         

In [26]:
%timeit Batch.stack(batches)

91.5 µs ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865],
               [-0.1372,  0.0257, -0.1627],
               [ 0.2138, -0.8140,  0.1865]]),
    x: Batch(
           c: tensor([[ 1.8700,  1.2475, -2.5124,  0.7783],
                      [ 0.7391,  0.2616,  0.2534, -0.1498],
                      [ 0.4851,  1.7314, -0.3190,  1.2785],
                      [ 1.8700,  1.2475, -2.5124,  0.7783],
                      [ 0.7391,  

In [28]:
%timeit Batch.cat(batches)

171 µs ± 5.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

501 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
